In [ ]:
# This whole notebook has been written while following along other competitors' notebooks
# to help teach me the common concepts, tools, and procedures when tackling a data analysis problem
%reset -sf

In [ ]:
import collections
import random
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
input_file = '/kaggle/input/hashcode-2021-oqr-extension/hashcode.in'
output_file = 'submission.csv'

def parse_input(input_file):
    with open(input_file) as f:
        arr = f.readlines()
    arr = arr[::-1]
    line = arr.pop().split()
    duration, num_intersections, num_streets, num_cars, scoring = [int(x) for x in line]
    streets = {} #this instantiates a blank dictionary
    
    for _ in range(num_streets):
        line = arr.pop().split()
        start = int(line[0]) # The indexing is based on the info in competition description
        end = int(line[1])
        street_name = line[2]
        length = int(line[3])
        streets[street_name] = start, end, length
        
    cars = []
    for idx in range(num_cars):
        line = arr.pop().split()
        sequence = line[1:] # [1:] because the first word is the length of the sequence
        cars.append(sequence)
        
    return duration, num_intersections, num_streets, num_cars, scoring, streets, cars
    
    # Each element in 'cars' is an array of all the 'street_name' that the car traverses
    

In [ ]:
duration, num_intersections, num_streets, num_cars, scoring, streets, cars = parse_input(input_file)
    
duration, num_intersections, num_streets, num_cars, scoring

# **Data Analysis**

In [ ]:
distance_distribution = []

for car in cars:
    distance = 0
    for street_name in car:
        distance += streets[street_name][-1] # street_name: (start, end, length) (as described in directions)
        
    distance_distribution.append(distance)

In [ ]:
#now we graph the distribution of distances traveled

plt.figure(figsize=(14,2))
plt.hist(distance_distribution, bins=100)
plt.axvline(duration, color='r')
plt.title('Distribution of distances to travel for each vehicle')
plt.show()
plt.close()

# Upper bound estimate
* First we parsed the data into something more easily discernible
* Then we analysed the distribution of distances the cars will travel
* Next up is figuring out the max potential aka the Upper Bound

To discover this we assume cars don't wait at every intersection

In [ ]:
impossible_cars = 0 # number of cars that can't finish
total_distance_required = 0 #total distance to be traveled by the car

for car in cars:
    distance = 0
    for street_name in car[1:]: # [1:] because the car doesn't need to travel the first street
        distance += streets[street_name][-1]
        
    if distance > duration:
        impossible_cars += 1
    else:
        total_distance_required += distance
        
# for completing before the end of the simulation
completion_score = scoring * (num_cars-impossible_cars)
# bonus for completing early
time_bonus = duration * (num_cars-impossible_cars) - total_distance_required
total_score = completion_score + time_bonus

total_distance_required, impossible_cars, time_bonus, completion_score, total_score

# Sample Solution
In this sample solution, we assign the duration of each green light to one, if there is a car coming from that direction

In [ ]:
# this is to discover which intersection each road leads to
street_dest = {}
street_source = {}
for street_name, (start,end,length) in streets.items():
    street_dest[street_name] = start
    street_source[street_name] = end

In [ ]:
# for each intersection, count the amount of traffic from each incoming street

incoming_count = collections.defaultdict(collections.Counter)
for car in cars:
    for street_name in car:
        incoming_count[street_dest[street_name]][street_name] += 1

In [ ]:
schedules = []
for i in range(num_intersections):
    total_count = sum(incoming_count[i].values()) # number of incoming streets
    num_incoming = len(incoming_count[i]) # amount of incoming traffic
    
    arr = list(incoming_count[i].items())
    random.shuffle(arr) # shuffle the incoming streets for a randomised solution
    
    cycle = []
    for incoming, count in arr:
        time_fraction = 1 # the green light duration is always one
        cycle.append([incoming, time_fraction])
    schedules.append(cycle)

# Parse solution into submission

In [ ]:
res = []
res.append([len(schedules)])
for i, cycle in enumerate(schedules):
    if not cycle:
        res[0][0] -= 1
        continue
    res.append([i])
    res.append([len(cycle)])
    for incoming, time_fraction in cycle:
        res.append([incoming, time_fraction])

In [ ]:
result_string = "\n".join(" ".join([str(x) for x in row]) for row in res)
# print(result_string)

In [ ]:
with open("submission.csv", "w") as text_file:
    text_file.writ(result_string)